# CS 5683 - Big Data Analytics
## Assignment - 1: Intro. to Spark and RDD

###### Use Google Colab to use this notebook
###### Let's setup Spark first

In [4]:
!pip install pyspark

     |████████████████████████████████| 204.7MB 66kB/s 
     |████████████████████████████████| 204kB 46.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044184 sha256=7e98f6997c5b02258774b678ad22bfe2f991a96a4d308530bc207ee856252f83
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


###### Import required libraries now

In [15]:
import sys
 
from pyspark import SparkContext, SparkConf

###### Let's initialize Spark context now

In [6]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark Word Count Exmaple")

###### Follow the tutorial to mount your Google Drive. Give mounted Drive paths below

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
# Give **.txt FILE PATHS** here
file1 = '/content/drive/My Drive/BigDataAnalytics/Assignment1/file1.txt'
file2 = '/content/drive/My Drive/BigDataAnalytics/Assignment1/file2.txt'

# USE THESE FILES as input(s) FOR ALL BELOW QUESTIONS

### Example Spark program

In [9]:
# Example Spark application for a simple wordcount
# What is wordcount? 
    # Given a file, count the frequency of all words appearing in that file
    
# Step-1: Read the required file. In our case it is file1 or file2.
# NOTE: We do not need to initialize SparkContext as only one SparkContext can be initialized in one notebook
fileRDD = sc.textFile(file1)
#wordsRDD = fileRDD.flatMap(lambda line: line.split("."))
# Step-2: 
    # Each line in our file(s) is a sentence. So, we need to split the sentence with ' ' to get words
    # Using map() will return RDD[list]. But we need RDD[string]. So we use flatMap()
wordsRDD = fileRDD.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

# Step-3: For each input, we will make (K,V) pair, where K is the word and V is 1
pairRDD = wordsRDD.map(lambda word: (word,1))

# Step-4: Now we have to sum all 1's of each word
# NOTE: A word may present in multiple data partitions. So we use reduceByKey() to group by key and perform sum
countRDD = pairRDD.reduceByKey(lambda a,b: a+b)

#Step-5: Save results in a text file
countRDD.saveAsTextFile('/content/drive/My Drive/BigDataAnalytics/Assignment1/example') # <----------- GIVE FILE PATH

### Question - 1 (10 points) 

Write a Spark application that counts number of words that ends with each letter in file1. That is for each letter count the number of (non-unique) words that ends with the specific letter. You can **ignore** letter cases (consider the given text contains only lower-case letters). Also, you can **ignore** words that end with non-alphabetic letters. **Sort the output in alphabetical order**

Example Output:
('a', 500)
('B', 100)

which means that the given input has 500 words that end with the letter 'a' and 100 words that end with letter 'B'.
NOTE: The output is sorted, the application counts duplicate words also, and the application is not case-sensitive

In [10]:
# YOUR CODE for Question-1 HERE 
#Filter the null value from word list 
wordsRDD2=wordsRDD.filter (lambda x: x!='')
#convert all character into lowercase
wordsRDD3=wordsRDD2.map (lambda x: x.lower())
#get the last character of the word and map them with 1
lastCharRDD = wordsRDD3.map(lambda word: (word[-1],1))
# sum all 1 reducebykey function
countLastChar = lastCharRDD.reduceByKey(lambda a,b: a+b)
countLastCharAcc=countLastChar.sortByKey(ascending=True)
countLastCharAcc.saveAsTextFile('/content/drive/My Drive/BigDataAnalytics/Assignment1/question1')


In [ ]:
# PRINT THE OUTPUT HERE

### Question - 2 (10 points)

Write a Spark application that counts the **unique** number of words that have **n-letters** in file1. That is count the number of **unique** 1-letter words, 2-letter words, 3-letter words, etc. You can **include** all non-alphabetic letters for this application.

Example Output:
(1, 100)
(2, 700)
(3, 1500)

which means that the input file for the Spark application has **100 unique 1-letter words** , **700 unique 2-letter words** , and **1500 uniqe 3-letter words**

In [ ]:
# YOUR CODE for Question-2 HERE

In [14]:
#get the unique vlue from the RDD
DistinctWordRDD=wordsRDD3.distinct()
#Get the length of unque word and map them with 1 
WordLenRDD = DistinctWordRDD.map(lambda word: (len(word),1))
#sum all 1 from map 
countRDD4 = WordLenRDD.reduceByKey(lambda a,b: a+b)
#sort the order
countRDD5=countRDD4.sortByKey(ascending=True)
#save the output on file
countRDD5.saveAsTextFile('/content/drive/My Drive/BigDataAnalytics/Assignment1/question2')

In [ ]:
# PRINT THE OUTPUT HERE

### Question - 3 (10 points)

Write a Spark aplication that outputs wordcount from two files: file1 and file2. That is count the number of occurances of words from two files. You can **ignore** letter cases (consider the given input files contain only lower-case letters). Also, you can **ignore** that is not present in both files.

Example Output:
(big, (10, 20))
(Data, (30, 50))

which means the word "big" appears 10 times in file-1 and 20 times in file-2 and the word "Data" appears 30 times in file-1 and 50 times in file-2

In [ ]:
# YOUR CODE for Question-3 HERE

In [16]:
#get the second file
fileRDD2 = sc.textFile(file2)
#split the line using whitespace
wordsRDDf2 = fileRDD2.flatMap(lambda line: line.split(" ")) 
#convert all character into lowercase
wordslower=wordsRDDf2.map (lambda x: x.lower())
# Step-3: For each input, we will make (K,V) pair, where K is the word and V is 1
pairRDDf2 = wordslower.map(lambda word: (word,1))

countRDDf2 = pairRDDf2.reduceByKey(lambda a,b: a+b)
#join 2 RDD from file1 and file2
finalRDD=countRDD.join(countRDDf2)
#Step-3: For each input, we will make (K,(v1,v2)) pair, where K is the word and v1 is word frequency in file1 and v2 is word frequency in file2
finalRDD2=finalRDD.reduceByKey(lambda a,b: (a,b))
finalRDD2.saveAsTextFile('/content/drive/My Drive/BigDataAnalytics/Assignment1/question3')

In [ ]:
# PRINT THE OUTPUT HERE

### WHAT TO TURN-IN IN CANVAS

1. Complete questions 1,2, and 3
2. Download this completed python notebook as .ipynb
3. Upload it in Canvas assignment

# Due Date: Sept. 3 at 11:59pm